In [1]:
import getopt
import sys

if '-i' in sys.argv:
    myopts, args = getopt.getopt(sys.argv[1:],"i:c:")

    for o, a in myopts:
        if o == '-i':
            i=int(a)
        elif o == '-c':
            n_cores=int(a)
        else:
            print("Usage: %s -i iteration -n_cores n_cores" % sys.argv[0])

    # Display input and output file name passed as the args
    print ("Running: %d with n_cores: %d" % (i, n_cores) )
else:
    i = 0
    n_cores=11
    print('Running locally')

# set theano flags
import os
os.environ["THEANO_FLAGS"] = "compiledir=./theano/%s/" %(str(i))

Running locally


In [2]:
import pandas as pd
import pymc3 as pm
import numpy as np
import glob
import os
import pickle as pkl

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
# code for smooth switch point:
# https://gist.github.com/junpenglao/f7098c8e0d6eadc61b3e1bc8525dd90d
import theano.tensor as tt
from pymc3.distributions.transforms import ElemwiseTransform, Transform

class Ordered(ElemwiseTransform):
    name = "ordered"

    def backward(self, y):
        out = tt.zeros(y.shape)
        out = tt.inc_subtensor(out[0], y[0])
        out = tt.inc_subtensor(out[1:], tt.exp(y[1:]))
        return tt.cumsum(out)

    def forward(self, x):
        out = tt.zeros(x.shape)
        out = tt.inc_subtensor(out[0], x[0])
        out = tt.inc_subtensor(out[1:], tt.log(x[1:] - x[:-1]))
        return out

    def forward_val(self, x, point=None):
        x, = draw_values([x], point=point)
        return self.forward(x)

    def jacobian_det(self, y):
        return tt.sum(y[1:])

ordered = Ordered()


class Composed(Transform):
    def __init__(self, transform1, transform2):
        self._transform1 = transform1
        self._transform2 = transform2
        self.name = '_'.join([transform1.name, transform2.name])

    def forward(self, x):
        return self._transform2.forward(self._transform1.forward(x))

    def forward_val(self, x, point=None):
        return self.forward(x)

    def backward(self, y):
        return self._transform1.backward(self._transform2.backward(y))

    def jacobian_det(self, y):
        y2 = self._transform2.backward(y)
        det1 = self._transform1.jacobian_det(y2)
        det2 = self._transform2.jacobian_det(y)
        return det1 + det2
    
def logistic(L, x0, k=50, t_=np.linspace(0., 1., 1000)):
    x0 = x0*(t_.max()-t_.min()) + t_.min()  # scale x0 to t_
    return L/(1+tt.exp(-k*(t_-x0)))

In [4]:
def sample_single_model(df, model_n, distribution, dep_var='rate', n_cores=15):

    with pm.Model() as model:
        mu = np.log(df[dep_var].mean())
        sd = np.log(df[dep_var].std())
        tune_steps = 500
        init = 'jitter+adapt_diag'

        if model_n == 1:
            # no change
            intercept = pm.Normal('intercept', mu=mu, sd=sd)
            ev = np.exp(intercept)

        elif model_n == 2:
            # gradient along pc axis 1
            intercept = pm.Normal('intercept', mu=mu, sd=sd)
            beta_pca_1 = pm.Normal('beta_pca_1', mu=0, sd=3)
            ev = np.exp(intercept + beta_pca_1*df['pc1_mm_norm'].values)

        elif model_n == 3:
            intercept = pm.Normal('intercept', mu=mu, sd=sd)
            # 3 sectors along pc axis 1
            delta_center_1 = pm.Normal('delta_center_1', mu=0, sd=3)
            delta_center_3 = pm.Normal('delta_center_3', mu=0, sd=3)

            ev = np.exp(intercept + \
                        delta_center_1*((df['pc1_mm_perc'].values<0.333).astype(int)) + \
                        delta_center_3*((df['pc1_mm_perc'].values>0.667).astype(int)))

        elif model_n == 4:
            intercept = pm.Normal('intercept', mu=mu, sd=sd)
            # gradient along pc axis 1+2+3
            beta_pca_1 = pm.Normal('beta_pca_1', mu=0, sd=3)
            beta_pca_2 = pm.Normal('beta_pca_2', mu=0, sd=3)
            beta_slice = pm.Normal('beta_slice', mu=0, sd=3)

            ev = tt.exp(intercept + beta_pca_1*df['pc1_mm_norm'].values + \
                        beta_pca_2*df['pc2_mm_norm'].values + \
                        beta_slice*df['slice_mm_norm'].values)

#         elif model_n == 5:
#             # cut-offs estimated, smoothness of cut-off estimated
#             nbreak = 3
#             lambdad = pm.Normal('lambdad', 0, sd=1, shape=3-1)
#             k = pm.HalfNormal('k', 20)  # always assume positive
#             trafo = Composed(pm.distributions.transforms.LogOdds(), Ordered())
#             b = pm.Beta('b', 4., 4., shape=nbreak-1, transform=trafo,
#                         testval=[0.33, 0.67])
#             ev = np.exp(intercept + logistic(lambdad[0], b[0], k=-k, t_=df['pc1_mm_norm'].values) +
#                                     logistic(lambdad[1], b[1], k=k, t_=df['pc1_mm_norm'].values))
#             tune_steps = 1500
            
        elif model_n == 6:
            # cut-offs estimated in 3D, smoothness of cut-off estimated
            beta_pca_1 = pm.HalfCauchy('beta_pca_1', 1)
            beta_pca_2 = pm.HalfCauchy('beta_pca_2', 1)
            beta_slice = pm.HalfCauchy('beta_slice', 1)
            
            k = pm.Laplace('smoothness', 500, 5) #pm.TruncatedNormal('smoothness', mu=100, sd=10, lower=0)
            trafo = Composed(pm.distributions.transforms.LogOdds(), Ordered())
#             lambdad = pm.Normal('lambdad', 0, sd, shape=3-1)
            lambda0 = pm.Normal('lambda0', mu=mu, sd=sd, shape=1)
            lambdas = pm.Normal('lambdas', mu=0, sd=sd, shape=2)
    
            b = pm.Beta('b', 4., 4., shape=2, transform=trafo, testval=[0.33, 0.67])

            # projection on new axis
#            beta_slice = pm.Deterministic('beta_slice', np.sqrt(1**2-beta_pca_1**2-beta_pca_2**2))
            O_vec = beta_pca_1*df['pc1_mm_norm'].values + \
                    beta_pca_2*df['pc2_mm_norm'].values + \
                    beta_slice*df['slice_mm_norm'].values
            # standardize new axis
            O_vec = (O_vec-O_vec.mean())/(O_vec.std())
            
#             ev = tt.exp(intercept + logistic(lambdad[0], b[0], k=k, t_=O_vec) +
#                                     logistic(lambdad[1]-lambdad[0], b[1], k=k, t_=O_vec))
            ev = tt.exp(lambda0 + \
                        logistic(lambdas[0], b[1], k=k, t_=O_vec) + \
                        logistic(lambdas[1]-lambdas[0], b[1], k=k, t_=O_vec))
                        
#                         lambdas[0]/(1+tt.exp(-k*(b[0]-t_))) + \
#                         lambdas[1]/(1+tt.exp(-k*(t_-b[0]))) + \
#                         (lambdas[2]-lambdas[1])/(1+tt.exp(-k*(t_-b[1])))))  # intercept section 3
            tune_steps = 500
            
#         elif model_n == 7:
#             # cut-offs estimated in 3D, smoothness of cut-off estimated
#             beta_pca_1 = pm.HalfNormal('beta_pca_1', sd=3)
#             beta_pca_2 = pm.HalfNormal('beta_pca_2', sd=3)
#             beta_slice = pm.HalfNormal('beta_slice', sd=3)
#             k = pm.HalfNormal('smoothness', sd=3)
#             trafo = Composed(pm.distributions.transforms.LogOdds(), Ordered())
#             b = pm.Beta('b', 4., 4., shape=2, transform=trafo, testval=[0.33, 0.67])
#             lambdad = pm.Normal('lambdad', mu=0, sd=1, shape=3-1)
            
#             # projection on new axis
#             O_vec = beta_pca_1*df['pc2.1'].values + \
#                     beta_pca_2*df['pc2.2'].values + \
#                     beta_slice*df['pc2.3'].values
#             # standardize new axis
#             O_vec = (O_vec-O_vec.mean())/(O_vec.std())
        
#             ev = np.exp(intercept + intercept*logistic(lambdad[0], b[0], k=k, t_=O_vec) +
#                                     intercept*logistic(lambdad[1], b[1], k=k, t_=O_vec))
#             tune_steps = 500

        # define likelihood
        if distribution == 'poisson':
            likelihood = pm.Poisson('y', mu=ev, observed=df[dep_var])
        else:
            alpha = pm.HalfCauchy('alpha', beta=2)
            likelihood = pm.NegativeBinomial('y', mu=ev, alpha=alpha, observed=df[dep_var])

        model.name = str(model_n) + '_' + distribution
        traces = pm.sample(cores=n_cores, tune=tune_steps, init=init)
        
        return model, traces


In [5]:
# load data
df = pd.read_pickle('./data_fwhm-0.3.pkl')
subjects = df.subject_id.unique()
stains = df.stain.unique()
models = [6,4,3,2,1]
distributions = ['poisson']#, 'negativebinomial']

output_dir = './models_local_advi_adapt'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [6]:
import itertools
all_combs = list(itertools.product(subjects, stains))

In [7]:
for i in [0,1,2,3,4,5,6,7,8,9,10]:
    subject, stain = all_combs[i]
    df_to_run = df.loc[(df.subject_id==subject) & (df.stain==stain),:]
    
    for model_n in models:
        for distribution in distributions:
            print('Subject {}, stain {}, model {}, distribution {}'.format(subject, stain, model_n, distribution))
            trace_fn = os.path.join(output_dir, 'sub-{}_stain-{}_model-{}_distribution-{}_type-traces.pkl').format(subject, stain, model_n, distribution)
            model_fn = os.path.join(output_dir, 'sub-{}_stain-{}_model-{}_distribution-{}_type-model.pkl').format(subject, stain, model_n, distribution)
            if os.path.exists(trace_fn):
                continue

            model, traces = sample_single_model(df_to_run, model_n, distribution, n_cores=n_cores)
            with open(model_fn, 'wb') as f:
                pkl.dump(model, f)
            with open(trace_fn, 'wb') as f:
                pkl.dump(traces, f)

Subject 13095, stain CALR, model 6, distribution poisson


Auto-assigning NUTS sampler...
Initializing NUTS using advi+adapt_diag...
Average Loss = 10,971:  11%|█         | 22494/200000 [01:10<08:58, 329.77it/s]
Convergence achieved at 22500
Interrupted at 22,499 [11%]: Average Loss = 13,476
Multiprocess sampling (11 chains in 11 jobs)
NUTS: [b_logodds, lambdas, lambda0, smoothness, beta_slice, beta_pca_2, beta_pca_1]
Sampling 11 chains:  94%|█████████▍| 10377/11000 [36:24<04:01,  2.58draws/s] 
There were 6 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.04172245731169198, but should be close to 0.8. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.5399983292181264, but should be close to 0.8. Try to increase the number of tuning steps.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
There were 2 divergences after tuning. Increase `target_accept`

Subject 13095, stain CALR, model 4, distribution poisson


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (11 chains in 11 jobs)
NUTS: [beta_slice, beta_pca_2, beta_pca_1, intercept]
Process worker_chain_8:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 251, in _bootstrap
    util._run_after_forkers()
  File "/opt/conda/lib/python3.6/multiprocessing/util.py", line 128, in _run_after_forkers
    items = list(_afterfork_registry.items())
  File "/opt/conda/lib/python3.6/weakref.py", line 209, in items
    if self._pending_removals:
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
pm.traceplot(traces)

In [ ]:
trace_df = pm.trace_to_dataframe(traces)
import seaborn as sns
sns.pairplot(trace_df)